In [ ]:
#change current working dictonary, this file should in PathomicFusion folder 
import os
os.chdir("PathomicFusion")

In [ ]:
print(os.getcwd())
!ls
!ls data
!ls data/TCGA_GBMLGG
!ls data/TCGA_GBMLGG/gbmlgg

In [ ]:
import pickle

split_path = "data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_patches_512_0_0_0.pkl"  

with open(split_path, "rb") as f:
    splits = pickle.load(f)


print(type(splits))
print(list(splits.keys())[:10])

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import importlib

packages = {
    "torch": "torch",
    "torchvision": "torchvision",
    "scikit-learn": "sklearn",
    "opencv-python": "cv2",
    "tensorboard": "tensorboard",
    "scipy": "scipy",
    "lifelines": "lifelines",
}

for pip_name, import_name in packages.items():
    try:
        mod = importlib.import_module(import_name)
        version = getattr(mod, "__version__", "no __version__ attr")
        print(f"[OK] {pip_name:15s}  (import as '{import_name}')  version: {version}")
    except ImportError:
        print(f"[MISSING] {pip_name:15s}  (import as '{import_name}')")

In [ ]:
import sys
!"{sys.executable}" -m pip install\
    opencv-python \
    tensorboard \
    lifelines

In [ ]:
!"{sys.executable}" -m pip install tables 

In [ ]:
!"{sys.executable}" -m pip install torch_geometric

In [ ]:
!"{sys.executable}" -m pip install \
    imbalanced-learn \
    pillow \

In [ ]:
!"{sys.executable}" -m pip install torchsummary

In [ ]:
# small test on omic alone 
#changes make in train_cv.py: 
#data_cv_path =  and in part 3 Sets-Up Main Loop of train_cv.py(change the code 
#(for k, data in data_cv_splits.items():) to 
# (all_splits = list(data_cv_splits.items()for k, data in all_splits[:1]: )
!"{sys.executable}" train_cv.py \
  --dataroot ./data/TCGA_GBMLGG \
  --checkpoints_dir ./checkpoints/TCGA_GBMLGG \
  --exp_name grad_15 \
  --task grad \
  --mode omic \
  --model_name omic \
  --niter 3 \
  --niter_decay 0 \
  --batch_size 64 \
  --lr 0.002 \
  --reg_type all \
  --init_type max \
  --weight_decay 5e-4 \
  --act LSM \
  --label_dim 3 \
  --gpu_ids -1 \
  --use_rnaseq 0 \
  --verbose 1

In [ ]:
# small test on pathomic 
#changes make in train_cv.py: 
#data_cv_path =  and in part 3 Sets-Up Main Loop of train_cv.py(change the code 
#(for k, data in data_cv_splits.items():) to 
# (all_splits = list(data_cv_splits.items()for k, data in all_splits[:1]: )



!"{sys.executable}" train_cv.py \
    --exp_name grad_15_small  \
    --task grad \
    --mode pathomic \
    --model_name pathomic_fusion \
    --niter 2 \
    --niter_decay 0 \
    --batch_size 4 \
    --lr 0.0001 \
    --beta1 0.5 \
    --fusion_type pofusion \
    --mmhid 32 \
    --use_bilinear 1 \
    --use_vgg_features 1 \
    --gpu_ids -1 \
    --path_gate 1 \
    --omic_gate 1 \
    --omic_scale 2 \
    --act LSM \
    --label_dim 3 \
    --path_dim 32 \
    --reg_type none


In [ ]:
pkl_path = "./data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_patches_512_0_0_1.pkl"
data_cv = pickle.load(open(pkl_path, "rb"))
cv_splits = data_cv["cv_splits"]

fold1 = cv_splits[1]
names = fold1["train"]["x_patname"]

print("Example x_patname (first 10):")
for n in names[:10]:
    print(n, "len =", len(n))

print("\nCompass index example (first 10):")
import pandas as pd
compass = pd.read_csv("./data/TCGA_GBMLGG/compass_GMBLGG.csv")
if "TCGA ID" in compass.columns:
    compass = compass.set_index("TCGA ID")
elif "TCGA_ID" in compass.columns:
    compass = compass.set_index("TCGA_ID")

for n in compass.index[:10]:
    print(n, "len =", len(n))


In [ ]:
#check id of embedding file and the data in pickle file 

pkl_path = "./data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_0_0_0.pkl"
data_cv = pickle.load(open(pkl_path, "rb"))
cv_splits = data_cv["cv_splits"]


all_tcga_ids = set()
for k, fold in cv_splits.items():
    for split in ["train", "test"]:
        all_tcga_ids.update(fold[split]["x_patname"])

print("Total unique TCGA IDs in GBMLGG splits:", len(all_tcga_ids))
print("Example from splits:", list(all_tcga_ids)[:10])


compass = pd.read_csv("./data/TCGA_GBMLGG/compass_GMBLGG.csv")

if "TCGA ID" in compass.columns:
    compass_ids = compass["TCGA ID"].astype(str)
elif "TCGA_ID" in compass.columns:
    compass_ids = compass["TCGA_ID"].astype(str)
else:
    compass_ids = compass.index.to_series().astype(str)


compass_core_ids = set(compass_ids.str[:12].tolist())

print("Total unique core IDs in COMPASS:", len(compass_core_ids))
print("Example from COMPASS core IDs:", list(compass_core_ids)[:10])


overlap = all_tcga_ids & compass_core_ids
print("\nIntersection size:", len(overlap))
print("Example intersection IDs:", list(overlap)[:10])


In [ ]:

pkl_path = "./data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_patches_512_0_0_1.pkl"
data_cv = pickle.load(open(pkl_path, "rb"))
cv_splits = data_cv["cv_splits"]


all_tcga_ids = set()
for k, fold in cv_splits.items():
    for split in ["train", "test"]:
        all_tcga_ids.update(fold[split]["x_patname"])

print("Total unique TCGA IDs in GBMLGG splits:", len(all_tcga_ids))
print("Example from splits:", list(all_tcga_ids)[:10])


compass = pd.read_csv("./data/TCGA_GBMLGG/compass_GMBLGG.csv")

if "TCGA ID" in compass.columns:
    compass_ids = compass["TCGA ID"].astype(str)
elif "TCGA_ID" in compass.columns:
    compass_ids = compass["TCGA_ID"].astype(str)
else:
    compass_ids = compass.index.to_series().astype(str)


compass_core_ids = set(compass_ids.str[:12].tolist())

print("Total unique core IDs in COMPASS:", len(compass_core_ids))
print("Example from COMPASS core IDs:", list(compass_core_ids)[:10])


overlap = all_tcga_ids & compass_core_ids
print("\nIntersection size:", len(overlap))
print("Example intersection IDs:", list(overlap)[:10])


In [ ]:

pd.read_csv("data/TCGA_GBMLGG/compass_GMBLGG.csv",index_col=False)
#value_counts('TCGA ID').values

In [ ]:


pkl_path = "./data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_patches_512_0_0_1_compass.pkl"

with open(pkl_path, "rb") as f:
    data = pickle.load(f)

list(data.keys())

In [ ]:
fold = list(data["cv_splits"].keys())[0]
data_fold = data["cv_splits"][fold]
list(data_fold.keys())

In [ ]:
X_omic_train = data_fold["train"]["x_omic"]
X_omic_test  = data_fold["test"]["x_omic"]

print("train count:", len(X_omic_train))
print("test count:", len(X_omic_test))

In [ ]:
#replace the omics in gbmlgg15cv_all_st_0_0_0.pkl to the compass embedding
from sklearn.preprocessing import StandardScaler


dataroot = "./data/TCGA_GBMLGG"


orig_pkl_path    = os.path.join(dataroot, "splits", "gbmlgg15cv_all_st_0_0_0.pkl")
compass_csv_path = os.path.join(dataroot, "compass_GMBLGG.csv")


out_pkl_path     = os.path.join(dataroot, "splits", "gbmlgg15cv_all_st_COMPASS12_nogradem1.pkl")


with open(orig_pkl_path, "rb") as f:
    data_cv = pickle.load(f)

cv_splits = data_cv["cv_splits"]
print("Loaded original splits from:", orig_pkl_path)
print("Folds:", list(cv_splits.keys()))


compass_df = pd.read_csv(compass_csv_path)
print("\nRaw compass_df shape:", compass_df.shape)
print("Raw columns (first 5):", compass_df.columns[:5].tolist())


if "Unnamed: 0" in compass_df.columns:
    compass_df = compass_df.drop(columns=["Unnamed: 0"])
    print("Dropped 'Unnamed: 0', new compass_df shape:", compass_df.shape)


if "TCGA ID" in compass_df.columns:
    compass_df = compass_df.set_index("TCGA ID")
elif "TCGA_ID" in compass_df.columns:
    compass_df = compass_df.set_index("TCGA_ID")

print("After set_index, compass_df shape:", compass_df.shape)
print("Example COMPASS index:", compass_df.index[:5].tolist())


compass_df = compass_df.copy()
compass_df["core_id"] = compass_df.index.to_series().astype(str).str[:12]
compass_core = compass_df.set_index("core_id")
print("Compass by core_id shape:", compass_core.shape)
print("Example compass core IDs:", compass_core.index[:5].tolist())

D = compass_core.shape[1]  
print("COMPASS embedding dim D =", D)


for k, fold in cv_splits.items():
    print(f"\n===== Processing fold {k} =====")

    for split in ["train", "test"]:
        names = np.array(fold[split]["x_patname"])  '
        g     = np.array(fold[split]["g"], dtype=float)

        
        names_core = np.array([str(pid)[:12] for pid in names])

       
        mask_compass = np.isin(names_core, compass_core.index)
       
        mask_grade   = (g != -1)
        
        final_mask   = mask_compass & mask_grade

        print(f"  {split}: keep {final_mask.sum()}/{len(final_mask)} "
              f"(has COMPASS & grade != -1)")

        if final_mask.sum() == 0:
            print(f"  [Warning] fold {k} {split} has 0 valid samples, making empty split.")
            
            fold[split]["x_patname"] = []
           
            x_path0 = np.array(fold[split]["x_path"][0])
            fold[split]["x_path"] = np.empty((0,) + x_path0.shape, dtype=x_path0.dtype)
            fold[split]["x_grph"] = []
            fold[split]["e"] = np.array([])
            fold[split]["t"] = np.array([])
            fold[split]["g"] = np.array([])
            fold[split]["x_omic"] = np.zeros((0, D), dtype=np.float32)
            continue

        
        names_kept      = names[final_mask]
        names_core_kept = names_core[final_mask]

        fold[split]["x_patname"] = list(names_kept)
        fold[split]["x_path"]    = np.array(fold[split]["x_path"])[final_mask]
        fold[split]["x_grph"]    = [g_ for g_, m in zip(fold[split]["x_grph"], final_mask) if m]
        fold[split]["e"]         = np.array(fold[split]["e"])[final_mask]
        fold[split]["t"]         = np.array(fold[split]["t"])[final_mask]
        fold[split]["g"]         = np.array(fold[split]["g"])[final_mask]

        
        X_omic = compass_core.loc[names_core_kept].values.astype(np.float32)  # (N_split, D)
        fold[split]["x_omic"] = X_omic

    
    train_omic = fold["train"].get("x_omic", None)
    if isinstance(train_omic, np.ndarray) and train_omic.shape[0] > 0:
        scaler = StandardScaler().fit(train_omic)
        fold["train"]["x_omic"] = scaler.transform(train_omic)

        test_omic = fold["test"].get("x_omic", None)
        if isinstance(test_omic, np.ndarray) and test_omic.shape[0] > 0:
            fold["test"]["x_omic"] = scaler.transform(test_omic)
    else:
        print(f"  [Warning] fold {k} has no valid train x_omic, skip scaling.")


data_cv["cv_splits"] = cv_splits

with open(out_pkl_path, "wb") as f:
    pickle.dump(data_cv, f)

print("\nSaved clean COMPASS+no(-1) splits to:", out_pkl_path)


In [ ]:

dataroot = "./data/TCGA_GBMLGG"


orig_pkl_path    = os.path.join(dataroot, "splits", "gbmlgg15cv_all_st_patches_512_0_0_1.pkl")
compass_csv_path = os.path.join(dataroot, "compass_GMBLGG.csv")


out_pkl_path     = os.path.join(dataroot, "splits", "gbmlgg15cv_all_st_patches_512_0_0_1_compass.pkl")


with open(orig_pkl_path, "rb") as f:
    data_cv = pickle.load(f)

cv_splits = data_cv["cv_splits"]
print("Loaded original splits from:", orig_pkl_path)
print("Folds:", list(cv_splits.keys()))


compass_df = pd.read_csv(compass_csv_path)
print("\nRaw compass_df shape:", compass_df.shape)
print("Raw columns (first 5):", compass_df.columns[:5].tolist())


if "Unnamed: 0" in compass_df.columns:
    compass_df = compass_df.drop(columns=["Unnamed: 0"])
    print("Dropped 'Unnamed: 0', new compass_df shape:", compass_df.shape)


if "TCGA ID" in compass_df.columns:
    compass_df = compass_df.set_index("TCGA ID")
elif "TCGA_ID" in compass_df.columns:
    compass_df = compass_df.set_index("TCGA_ID")

print("After set_index, compass_df shape:", compass_df.shape)
print("Example COMPASS index:", compass_df.index[:5].tolist())


compass_df = compass_df.copy()
compass_df["core_id"] = compass_df.index.to_series().astype(str).str[:12]
compass_core = compass_df.set_index("core_id")
print("Compass by core_id shape:", compass_core.shape)
print("Example compass core IDs:", compass_core.index[:5].tolist())

D = compass_core.shape[1]  
print("COMPASS embedding dim D =", D)


for k, fold in cv_splits.items():
    print(f"\n===== Processing fold {k} =====")

    for split in ["train", "test"]:
        names = np.array(fold[split]["x_patname"])  
        g     = np.array(fold[split]["g"], dtype=float)

        
        names_core = np.array([str(pid)[:12] for pid in names])

       
        mask_compass = np.isin(names_core, compass_core.index)
       
        mask_grade   = (g != -1)
        
        final_mask   = mask_compass & mask_grade

        print(f"  {split}: keep {final_mask.sum()}/{len(final_mask)} "
              f"(has COMPASS & grade != -1)")

        if final_mask.sum() == 0:
            print(f"  [Warning] fold {k} {split} has 0 valid samples, making empty split.")
            
            fold[split]["x_patname"] = []
           
            x_path0 = np.array(fold[split]["x_path"][0])
            fold[split]["x_path"] = np.empty((0,) + x_path0.shape, dtype=x_path0.dtype)
            fold[split]["x_grph"] = []
            fold[split]["e"] = np.array([])
            fold[split]["t"] = np.array([])
            fold[split]["g"] = np.array([])
            fold[split]["x_omic"] = np.zeros((0, D), dtype=np.float32)
            continue

        
        names_kept      = names[final_mask]
        names_core_kept = names_core[final_mask]

        fold[split]["x_patname"] = list(names_kept)
        fold[split]["x_path"]    = np.array(fold[split]["x_path"])[final_mask]
        fold[split]["x_grph"]    = [g_ for g_, m in zip(fold[split]["x_grph"], final_mask) if m]
        fold[split]["e"]         = np.array(fold[split]["e"])[final_mask]
        fold[split]["t"]         = np.array(fold[split]["t"])[final_mask]
        fold[split]["g"]         = np.array(fold[split]["g"])[final_mask]

        
        X_omic = compass_core.loc[names_core_kept].values.astype(np.float32)  # (N_split, D)
        fold[split]["x_omic"] = X_omic

    
    train_omic = fold["train"].get("x_omic", None)
    if isinstance(train_omic, np.ndarray) and train_omic.shape[0] > 0:
        scaler = StandardScaler().fit(train_omic)
        fold["train"]["x_omic"] = scaler.transform(train_omic)

        test_omic = fold["test"].get("x_omic", None)
        if isinstance(test_omic, np.ndarray) and test_omic.shape[0] > 0:
            fold["test"]["x_omic"] = scaler.transform(test_omic)
    else:
        print(f"  [Warning] fold {k} has no valid train x_omic, skip scaling.")


data_cv["cv_splits"] = cv_splits

with open(out_pkl_path, "wb") as f:
    pickle.dump(data_cv, f)

print("\nSaved clean COMPASS+no(-1) splits to:", out_pkl_path)

In [ ]:

dataroot = "./data/TCGA_GBMLGG"


orig_pkl_path    = os.path.join(dataroot, "splits", "gbmlgg15cv_all_st_patches_512_0_1_1.pkl")
compass_csv_path = os.path.join(dataroot, "compass_GMBLGG.csv")


out_pkl_path     = os.path.join(dataroot, "splits", "gbmlgg15cv_all_st_patches_512_0_1_1_compass.pkl")


with open(orig_pkl_path, "rb") as f:
    data_cv = pickle.load(f)

cv_splits = data_cv["cv_splits"]
print("Loaded original splits from:", orig_pkl_path)
print("Folds:", list(cv_splits.keys()))


compass_df = pd.read_csv(compass_csv_path)
print("\nRaw compass_df shape:", compass_df.shape)
print("Raw columns (first 5):", compass_df.columns[:5].tolist())


if "Unnamed: 0" in compass_df.columns:
    compass_df = compass_df.drop(columns=["Unnamed: 0"])
    print("Dropped 'Unnamed: 0', new compass_df shape:", compass_df.shape)


if "TCGA ID" in compass_df.columns:
    compass_df = compass_df.set_index("TCGA ID")
elif "TCGA_ID" in compass_df.columns:
    compass_df = compass_df.set_index("TCGA_ID")

print("After set_index, compass_df shape:", compass_df.shape)
print("Example COMPASS index:", compass_df.index[:5].tolist())


compass_df = compass_df.copy()
compass_df["core_id"] = compass_df.index.to_series().astype(str).str[:12]
compass_core = compass_df.set_index("core_id")
print("Compass by core_id shape:", compass_core.shape)
print("Example compass core IDs:", compass_core.index[:5].tolist())

D = compass_core.shape[1]  
print("COMPASS embedding dim D =", D)


for k, fold in cv_splits.items():
    print(f"\n===== Processing fold {k} =====")

    for split in ["train", "test"]:
        names = np.array(fold[split]["x_patname"])  
        g     = np.array(fold[split]["g"], dtype=float)

        
        names_core = np.array([str(pid)[:12] for pid in names])

       
        mask_compass = np.isin(names_core, compass_core.index)
       
        mask_grade   = (g != -1)
        
        final_mask   = mask_compass & mask_grade

        print(f"  {split}: keep {final_mask.sum()}/{len(final_mask)} "
              f"(has COMPASS & grade != -1)")

        if final_mask.sum() == 0:
            print(f"  [Warning] fold {k} {split} has 0 valid samples, making empty split.")
            
            fold[split]["x_patname"] = []
           
            x_path0 = np.array(fold[split]["x_path"][0])
            fold[split]["x_path"] = np.empty((0,) + x_path0.shape, dtype=x_path0.dtype)
            fold[split]["x_grph"] = []
            fold[split]["e"] = np.array([])
            fold[split]["t"] = np.array([])
            fold[split]["g"] = np.array([])
            fold[split]["x_omic"] = np.zeros((0, D), dtype=np.float32)
            continue

        
        names_kept      = names[final_mask]
        names_core_kept = names_core[final_mask]

        fold[split]["x_patname"] = list(names_kept)
        fold[split]["x_path"]    = np.array(fold[split]["x_path"])[final_mask]
        fold[split]["x_grph"]    = [g_ for g_, m in zip(fold[split]["x_grph"], final_mask) if m]
        fold[split]["e"]         = np.array(fold[split]["e"])[final_mask]
        fold[split]["t"]         = np.array(fold[split]["t"])[final_mask]
        fold[split]["g"]         = np.array(fold[split]["g"])[final_mask]

        
        X_omic = compass_core.loc[names_core_kept].values.astype(np.float32)  # (N_split, D)
        fold[split]["x_omic"] = X_omic

    
    train_omic = fold["train"].get("x_omic", None)
    if isinstance(train_omic, np.ndarray) and train_omic.shape[0] > 0:
        scaler = StandardScaler().fit(train_omic)
        fold["train"]["x_omic"] = scaler.transform(train_omic)

        test_omic = fold["test"].get("x_omic", None)
        if isinstance(test_omic, np.ndarray) and test_omic.shape[0] > 0:
            fold["test"]["x_omic"] = scaler.transform(test_omic)
    else:
        print(f"  [Warning] fold {k} has no valid train x_omic, skip scaling.")


data_cv["cv_splits"] = cv_splits

with open(out_pkl_path, "wb") as f:
    pickle.dump(data_cv, f)

print("\nSaved clean COMPASS+no(-1) splits to:", out_pkl_path)

In [ ]:


pkl_path_orig = "./data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_patches_512_0_0_0.pkl"
pkl_path_comp = "./data/TCGA_GBMLGG/splits/gbmlgg15cv_all_st_patches_512_0_0_0_compass.pkl"

with open(pkl_path_orig, "rb") as f:
    orig = pickle.load(f)
with open(pkl_path_comp, "rb") as f:
    comp = pickle.load(f)

orig_pd = orig["data_pd"]
comp_pd = comp["data_pd"]

print("orig columns:", orig_pd.columns)
print("comp columns:", comp_pd.columns)

print("\norig dtypes:")
print(orig_pd.dtypes)
print("\ncomp dtypes:")
print(comp_pd.dtypes)


In [ ]:
#test if the new compass pickle input works on omicnet


!"{sys.executable}" train_cv.py \
  --exp_name grad_compass_omic \
  --task grad \
  --mode omic \
  --model_name omic \
  --input_size_omic 1376 \
  --niter 2 \
  --niter_decay 0 \
  --batch_size 64 \
  --lr 0.002 \
  --reg_type none \
  --init_type max \
  --weight_decay 5e-4 \
  --act LSM \
  --label_dim 3 \
  --gpu_ids -1 \
  --use_rnaseq 0 \
  --verbose 1


In [ ]:
#test if the new compass pickle input works on  pathomic


!"{sys.executable}" train_cv.py \
  --exp_name grad_compass_omic \
  --task surv \
 --mode pathomic \
  --model_name pathomic_fusion \
  --niter 2 \
  --niter_decay 0 \
  --batch_size 8 \
  --lr 0.0001 \
  --beta1 0.5 \
  --reg_type none \
  --fusion_type pofusion \
  --mmhid 64 \
  --use_bilinear 0 \
  --use_vgg_features 1 \
  --gpu_ids -1 \
  --omic_gate 0 \
  --use_rnaseq 1 \
  --input_size_omic 1376 \
  --path_dim 32\
  --finetune 1 \



In [ ]:


!"{sys.executable}" train_cv.py \
  --exp_name surv_compass_omic_gates \
  --task surv \
 --mode pathomic \
  --model_name pathomic_fusion \
  --niter 2 \
  --niter_decay 0 \
  --batch_size 8 \
  --lr 0.0001 \
  --beta1 0.5 \
  --reg_type none \
  --fusion_type pofusion \
  --mmhid 64 \
  --use_bilinear 0 \
  --use_vgg_features 1 \
  --gpu_ids -1 \
  --omic_gate 0 \
  --use_rnaseq 1 \
  --input_size_omic 1376 \
  --path_dim 32\
  --finetune 1 \

In [ ]:
#train with 2 epochs 

!"{sys.executable}" train_cv.py \
  --exp_name surv_compass_omic_gates_bi\
  --task surv \
 --mode pathomic \
  --model_name pathomic_fusion \
  --niter 2 \
  --niter_decay 0 \
  --batch_size 8 \
  --lr 0.0001 \
  --beta1 0.5 \
  --reg_type none \
  --fusion_type pofusion \
  --mmhid 64 \
  --use_bilinear 1 \
  --use_vgg_features 1 \
  --gpu_ids -1 \
  --omic_gate 0 \
  --use_rnaseq 1 \
  --input_size_omic 1376 \
  --path_dim 32\
  --finetune 1 \

In [ ]:
#train with 10 epochs ,lr 0.00005

import sys

!"{sys.executable}" train_cv.py \
  --exp_name surv_compass_omic_gates_bi_ff_ver2\
  --task surv \
 --mode pathomic \
  --model_name pathomic_fusion \
  --niter 5 \
  --niter_decay 5 \
  --batch_size 8 \
  --lr 0.00005 \
  --beta1 0.5 \
  --reg_type none \
  --fusion_type pofusion \
  --mmhid 64 \
  --use_bilinear 1 \
  --use_vgg_features 1 \
  --gpu_ids -1 \
  --omic_gate 0 \
  --use_rnaseq 1 \
  --input_size_omic 1376 \
  --path_dim 32\
  --finetune 0 \

In [ ]:
#train with 10 epochs ,lr 0.0001


!"{sys.executable}" train_cv.py \
  --exp_name surv_compass_omic_gates_bi_ff\
  --task surv \
 --mode pathomic \
  --model_name pathomic_fusion \
  --niter 5 \
  --niter_decay 5 \
  --batch_size 8 \
  --lr 0.0001 \
  --beta1 0.5 \
  --reg_type none \
  --fusion_type pofusion \
  --mmhid 64 \
  --use_bilinear 1 \
  --use_vgg_features 1 \
  --gpu_ids -1 \
  --omic_gate 0 \
  --use_rnaseq 1 \
  --input_size_omic 1376 \
  --path_dim 32\
  --finetune 0 \

In [ ]:


!"{sys.executable}" train_cv.py \
  --exp_name surv_compass_pgo_gates \
  --task surv \
 --mode pathgraphomic \
  --model_name pathgraphomic_fusion \
  --niter 2 \
  --niter_decay 0 \
  --batch_size 8 \
  --lr 0.0001 \
  --beta1 0.5 \
  --reg_type none \
  --fusion_type pofusion_A \
  --mmhid 64 \
  --use_bilinear 1 \
  --use_vgg_features 1 \
  --gpu_ids -1 \
  --omic_gate 0 \
  --use_rnaseq 1 \
  --input_size_omic 1376 \
  --path_dim 32\
  --finetune 1 \
  --grph_scale 2 

In [ ]:
##network changed (replace SNN with compass embedding that have already be placed  in the input pickle file using above vode)
#changed from Pathomic Fsusion network.py
#if want to use, need to paste this back to the network.py file of the pathomic fusion before run the training command
#also need to edit the train_test.py if want to store the gate activation as pickle file 
##############################################################################
# Path + Omic
##############################################################################
class PathomicNet(nn.Module):
    def __init__(self, opt, act, k): 
        super(PathomicNet, self).__init__() 
        self.path_dim = opt.path_dim 
        self.omic_dim = opt.omic_dim 
        
        #####change here 
        #self.path_encoder = resnet50(pretrained=True) 
        self.omic_proj = nn.Linear(opt.input_size_omic, opt.omic_dim) 
        hidden_dim = 64 
        self.gate_mlp = nn.Sequential( nn.Linear(self.path_dim + self.omic_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, self.path_dim), nn.Sigmoid() ) 
       
        self.fusion = define_bifusion(fusion_type=opt.fusion_type, skip=opt.skip, use_bilinear=opt.use_bilinear, gate1=opt.path_gate, gate2=opt.omic_gate, dim1=opt.path_dim, dim2=opt.omic_dim, scale_dim1=opt.path_scale, scale_dim2=opt.omic_scale, mmhid=opt.mmhid, dropout_rate=opt.dropout_rate) self.classifier = nn.Sequential(nn.Linear(opt.mmhid, opt.label_dim)) 
        self.act = act 
        #####change here 
        #dfs_freeze(self.omic_net) 
        self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False) 
        self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)
        
        
        def forward(self, **kwargs): 
            path_vec = kwargs['x_path'] 
            
            x_omic = kwargs['x_omic'] 
            omic_vec = self.omic_proj(x_omic) 
            
            concat = torch.cat([path_vec, omic_vec], dim=1) 
            gate = self.gate_mlp(concat) 
            gated_path = gate * path_vec 
            
            self.last_gate = gate 
            features = self.fusion(gated_path, omic_vec) 
            hazard = self.classifier(features) 
            if self.act is not None: 
                hazard = self.act(hazard) 
                if isinstance(self.act, nn.Sigmoid): 
                    hazard = hazard * self.output_range + self.output_shift 
            return features, hazard


    def __hasattr__(self, name):
        if '_parameters' in self.__dict__:
            _parameters = self.__dict__['_parameters']
            if name in _parameters:
                return True
        if '_buffers' in self.__dict__:
            _buffers = self.__dict__['_buffers']
            if name in _buffers:
                return True
        if '_modules' in self.__dict__:
            modules = self.__dict__['_modules']
            if name in modules:
                return True
        return False


